In [1]:
import pandas as pd
from matplotlib import pyplot as plt

df = pd.read_csv("churn-data-3333.csv")

In [2]:
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()
num_df.head()

,account_length,area_code,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
0,128,415,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1
1,107,415,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1
2,137,415,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0
3,84,408,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2
4,75,415,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3


In [3]:
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()

# Categorical boolean mask
categorical_feature_mask = cat_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = cat_df.columns[categorical_feature_mask].tolist()

import numpy as np
#conData=np.log(0.00001 + 1)
conData=0
cat_df=cat_df.fillna(conData)
num_df=num_df.fillna(conData)
cat_df=cat_df.astype(str)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_df[categorical_cols] = cat_df[categorical_cols].apply(lambda col: le.fit_transform(col))

#cat_df[categorical_cols].head(10)


churn=cat_df['churn']
cat_df=cat_df.drop(['churn'], axis=1)
cat_df=cat_df.drop(['phone number'], axis=1)



#BOX-COX
#lemda=0.5
#num_df=(num_df**lemda)
#num_df=num_df-1
#num_df[num_df < 0]=0
#num_df=num_df.div(lemda)
#End BOX-COX
#Z-score
num_df=(num_df-num_df.min())/(num_df.std(ddof=0)) ##Z-score

#Log
#num_df=round(np.log(num_df.add(1)),2)
#num_df=num_df.replace([np.inf, -np.inf], np.nan)
#End Log

num_df=num_df.fillna(conData)

result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)

result_df_op=result_df
#result_df_op=result_df_op.drop(['churn'], axis=1)
#result_df_op=np.nan_to_num(result_df_op)

X=result_df_op
#y=result_df['churn'] 
y=churn

from sklearn.model_selection import train_test_split  
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.0, random_state=42)
X_train=X
X_test=X
y_train=y
y_test=y

In [4]:
X.shape

(3333, 19)

In [5]:
X_train=X
y_train=y
#Univariate Selection
from sklearn.feature_selection import SelectKBest, chi2
select_feature = SelectKBest(chi2, k=15).fit(X_train, y_train)
selected_features_df = pd.DataFrame({'Feature':list(X_train.columns),
                                     'Scores':select_feature.scores_})
selected_features_df.sort_values(by='Scores', ascending=False)

,Feature,Scores
17,international_plan,203.244178
15,number_customer_service_calls,122.233984
2,number_vmail_messages,45.346680
3,total_day_minutes,42.493620
5,total_day_charge,42.492734
18,voice_mail_plan,25.156959
6,total_eve_minutes,7.241018
8,total_eve_charge,7.239392
13,total_intl_calls,5.113190
14,total_intl_charge,4.233468


In [6]:
y_train=y
X_train=X
x_train_chi = select_feature.transform(X)
x_train_chi.shape

(3333, 15)

In [7]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB()

classifier.fit(X,y_train)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

2623 227 224 259
pod:  0.5362318840579711
pof:  0.07964912280701754
AUC:  0.7282913806254768


In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler

cv_method = RepeatedKFold(n_splits=5, 
                          n_repeats=3, 
                          random_state=999)

classifier=GaussianNB();
#params = {
#          "priors" : "None",
#          "var_smoothing" : 1e-9
#}
#create an list of var_smoothing to cross validate
#steps = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4]

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

grid = GridSearchCV(estimator=classifier, 
                     param_grid=params_NB, 
                     cv=cv_method,
                     verbose=1, 
                     scoring='accuracy', n_jobs=-1)

grid.fit(x_train_chi, y_train)

 
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 15 folds for each of 100 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.5s


{'var_smoothing': 0.0006579332246575676}
GaussianNB(priors=None, var_smoothing=0.0006579332246575676)
0.8804880488048805


[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:    9.5s finished


26710 23728 21590 27972
pod:  0.5643840038739357
pof:  0.4704389547563345
AUC:  0.5469725245588006


In [9]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler

#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=0.0006579332246575676)

classifier.fit(x_train_chi,y_train)


cv_method = RepeatedKFold(n_splits=10, 
                          n_repeats=3, 
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 5)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

2773 77 320 163
pod:  0.33747412008281574
pof:  0.027017543859649124
AUC:  0.6552282881115833
accuracy:  0.8808880888088809


In [1]:
 #accuracy=(tp+tn)/(tp+fn+fp+tn)
accuracy=(26836+27976)/(27976+22462+22726+26836)
print ('accuracy: ',accuracy)

accuracy:  0.54812


In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Specify parameters
c_values = list(np.arange(1, 10))


param_grid = [
    {'C': c_values, 'penalty': ['l1'], 'solver' : ['liblinear'], 'multi_class' : ['ovr']},    
    {'C': c_values, 'penalty': ['l2'], 'solver' : ['liblinear', 'newton-cg', 'lbfgs'], 'multi_class' : ['ovr']}
]
 

grid = GridSearchCV(LogisticRegression(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.7s


{'C': 1, 'multi_class': 'ovr', 'penalty': 'l2', 'solver': 'newton-cg'}
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)
0.8628862886288629


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:    3.6s finished


In [11]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)
classifier.fit(x_train_chi, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2774 76 381 102
pod:  0.2111801242236025
pof:  0.02666666666666667
AUC:  0.5922567287784679
accuracy:  0.8628862886288629


In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier  

# Specify parameters
c_values = list(np.arange(1, 10))
param_grid ={
   'n_neighbors': [3,5,11,19],
   'weights': ['uniform', 'distance'],
   'metric': ['euclidean', 'manhattan']
}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, verbose=1, cv=3, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'uniform'}
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')
0.8748874887488749


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:    1.2s finished


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm

# Specify parameters

Cs = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
gammas = [0.001, 0.01, 0.1, 1, 1e-3, 1e-4]
param_grid = {'C': Cs, 'gamma' : gammas}

tuned_parameters = [{'kernel': ['rbf', 'linear'], 
                     'gamma': [0.001, 0.01, 0.1, 1, 1e-3, 1e-4], 
                     'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]} ]

grid = GridSearchCV(svm.SVC(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 42 candidates, totalling 420 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 410.7min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 1993.6min


In [13]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform' )  
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)
accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


2832 18 392 91
pod:  0.18840579710144928
pof:  0.00631578947368421
AUC:  0.5910450038138825
accuracy:  0.876987698769877


In [14]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(n_neighbors=5)  
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


2829 21 414 69
pod:  0.14285714285714285
pof:  0.007368421052631579
AUC:  0.5677443609022557
accuracy:  0.8694869486948695


{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}


#clf = RandomForestClassifier()

# Choose some parameter combinations to try
#parameters = {'n_estimators': [4, 6, 9], 
#              'max_features': ['log2', 'sqrt','auto'], 
#              'criterion': ['entropy', 'gini'],
#              'max_depth': [2, 3, 5, 10], 
#              'min_samples_split': [2, 3, 5],
#              'min_samples_leaf': [1,5,8]
#             }

#model_params = {
#    'n_estimators': [50, 150, 250],
#    'max_features': ['sqrt', 0.25, 0.5, 0.75, 1.0],
#    'min_samples_split': [2, 4, 6]
#}

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(x_train_chi, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   38.3s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 864 out of 864 | elapsed:  3.5min finished


{'bootstrap': True, 'max_depth': 90, 'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 200}
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=90, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
0.9507950795079508


In [17]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(
bootstrap=True, class_weight=None, criterion='gini',
            max_depth=90, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2830 20 141 342
pod:  0.7080745341614907
pof:  0.007017543859649123
AUC:  0.8505284951509208
accuracy:  0.9516951695169517


In [18]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier()
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


2831 19 153 330
pod:  0.6832298136645962
pof:  0.006666666666666667
AUC:  0.8382815734989648


In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

param_grid = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
    }

grid = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=10, n_jobs=-1)

grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)


KeyboardInterrupt: 

In [19]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier(criterion='entropy', splitter='best')
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2717 133 130 353
pod:  0.7308488612836439
pof:  0.04666666666666667
AUC:  0.8420910973084886
accuracy:  0.9210921092109211


In [20]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2720 130 126 357
pod:  0.7391304347826086
pof:  0.0456140350877193
AUC:  0.8467581998474447


In [21]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

#classifier = GradientBoostingClassifier(max_features=None, max_depth=3, criterion='friedman_mse')
classifier = GradientBoostingClassifier()
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2810 40 131 352
pod:  0.7287784679089027
pof:  0.014035087719298246
AUC:  0.8573716900948022
accuracy:  0.9486948694869487


In [22]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

classifier = GradientBoostingClassifier()
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2809 41 128 355
pod:  0.7349896480331263
pof:  0.014385964912280702
AUC:  0.8603018415604229
accuracy:  0.9492949294929492


In [24]:
df_x_train_chi = pd.DataFrame(x_train_chi)
df_x_train_chi.head()

#FNN
def get_FNN_Predict(X2_train, X2_test, y2_train, y2_test):
    from keras.models import Sequential
    from keras.layers import Dense
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = X2_train.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #compile model using mse as a measure of model performance
    #model.compile(optimizer='adam', loss='mean_squared_error')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  
    from keras.callbacks import EarlyStopping
    #set early stopping monitor so the model stops training when it won't improve anymore
    early_stopping_monitor = EarlyStopping(patience=3)
    #train model
    model.fit(X2_train, y2_train, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
    y2_pred = model.predict(X2_test)
    print('y2_pred: ',y2_pred)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_FNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

print('tn_list: ',tn_list)
print ('fp_list: ',fp_list)
print ('fn_list: ',fn_list)
print ('tp_list: ',tp_list)


tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)
 


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 2399 samples, validate on 600 samples
Epoch 1/30
2399/2399 [==============================] - 1s 219us/step - loss: 0.4571 - acc: 0.8479 - val_loss: 0.4307 - val_acc: 0.8417
Epoch 2/30
2399/2399 [==============================] - 0s 70us/step - loss: 0.4088 - acc: 0.8587 - val_loss: 0.4256 - val_acc: 0.8417
Epoch 3/30
2399/2399 [==============================] - 0s 73us/step - loss: 0.3985 - acc: 0.8587 - val_loss: 0.4635 - val_acc: 0.8417
Epoch 4/30
2399/2399 [==============================] - 0s 72us/step - loss: 0.3977 - acc: 0.8587 - val_loss: 0.4432 - val_acc: 0.8417
Epoch 5/30
2399/2399 [==============================] - 0s 77us/step - loss: 0.3905 - acc: 0.8587 - val_loss: 0.3926 - val_acc: 0.8417
Epoch 6/30
2399/2399 [==============================] - 0s 70us/step - loss: 0.3685 - acc: 0.8595 - val_loss: 0.3923 - val_acc: 0.8500
Epoch 7/30
2399/2399 [

Train on 2399 samples, validate on 600 samples
Epoch 1/30
2399/2399 [==============================] - 1s 243us/step - loss: 0.4643 - acc: 0.8462 - val_loss: 0.4457 - val_acc: 0.8417
Epoch 2/30
2399/2399 [==============================] - 0s 58us/step - loss: 0.4099 - acc: 0.8583 - val_loss: 0.4694 - val_acc: 0.8400
Epoch 3/30
2399/2399 [==============================] - 0s 62us/step - loss: 0.4115 - acc: 0.8579 - val_loss: 0.4851 - val_acc: 0.8400
Epoch 4/30
2399/2399 [==============================] - 0s 58us/step - loss: 0.4068 - acc: 0.8587 - val_loss: 0.4313 - val_acc: 0.8417
Epoch 5/30
2399/2399 [==============================] - 0s 59us/step - loss: 0.4024 - acc: 0.8587 - val_loss: 0.4065 - val_acc: 0.8417
Epoch 6/30
2399/2399 [==============================] - 0s 62us/step - loss: 0.3916 - acc: 0.8591 - val_loss: 0.4135 - val_acc: 0.8417
Epoch 7/30
2399/2399 [==============================] - 0s 58us/step - loss: 0.3820 - acc: 0.8587 - val_loss: 0.4092 - val_acc: 0.8417
Epoch 8

Train on 2399 samples, validate on 600 samples
Epoch 1/30
2399/2399 [==============================] - 1s 259us/step - loss: 0.4813 - acc: 0.8474 - val_loss: 0.4805 - val_acc: 0.8417
Epoch 2/30
2399/2399 [==============================] - 0s 65us/step - loss: 0.4185 - acc: 0.8587 - val_loss: 0.4869 - val_acc: 0.8417
Epoch 3/30
2399/2399 [==============================] - 0s 63us/step - loss: 0.4143 - acc: 0.8595 - val_loss: 0.4288 - val_acc: 0.8417
Epoch 4/30
2399/2399 [==============================] - 0s 63us/step - loss: 0.3950 - acc: 0.8587 - val_loss: 0.4148 - val_acc: 0.8417
Epoch 5/30
2399/2399 [==============================] - 0s 66us/step - loss: 0.3758 - acc: 0.8612 - val_loss: 0.4134 - val_acc: 0.8417
Epoch 6/30
2399/2399 [==============================] - 0s 72us/step - loss: 0.3739 - acc: 0.8587 - val_loss: 0.4014 - val_acc: 0.8433
Epoch 7/30
2399/2399 [==============================] - 0s 65us/step - loss: 0.3740 - acc: 0.8579 - val_loss: 0.4049 - val_acc: 0.8433
Epoch 8

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 1s 298us/step - loss: 0.4293 - acc: 0.8583 - val_loss: 0.4218 - val_acc: 0.8417
Epoch 2/30
2400/2400 [==============================] - 0s 62us/step - loss: 0.4004 - acc: 0.8583 - val_loss: 0.4123 - val_acc: 0.8417
Epoch 3/30
2400/2400 [==============================] - 0s 63us/step - loss: 0.3828 - acc: 0.8608 - val_loss: 0.3966 - val_acc: 0.8467
Epoch 4/30
2400/2400 [==============================] - 0s 63us/step - loss: 0.3723 - acc: 0.8625 - val_loss: 0.3906 - val_acc: 0.8417
Epoch 5/30
2400/2400 [==============================] - 0s 61us/step - loss: 0.3518 - acc: 0.8646 - val_loss: 0.3795 - val_acc: 0.8550
Epoch 6/30
2400/2400 [==============================] - 0s 68us/step - loss: 0.3411 - acc: 0.8708 - val_loss: 0.3484 - val_acc: 0.8700
Epoch 7/30
2400/2400 [==============================] - 0s 67us/step - loss: 0.3255 - acc: 0.8700 - val_loss: 0.3454 - val_acc: 0.8633
Epoch 8

...

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 1s 366us/step - loss: 0.4569 - acc: 0.8575 - val_loss: 0.4560 - val_acc: 0.8417
Epoch 2/30
2400/2400 [==============================] - 0s 68us/step - loss: 0.4214 - acc: 0.8583 - val_loss: 0.4356 - val_acc: 0.8417
Epoch 3/30
2400/2400 [==============================] - 0s 70us/step - loss: 0.3955 - acc: 0.8583 - val_loss: 0.4079 - val_acc: 0.8417
Epoch 4/30
2400/2400 [==============================] - 0s 65us/step - loss: 0.3861 - acc: 0.8612 - val_loss: 0.3987 - val_acc: 0.8467
Epoch 5/30
2400/2400 [==============================] - 0s 68us/step - loss: 0.3643 - acc: 0.8658 - val_loss: 0.4164 - val_acc: 0.8467
Epoch 6/30
2400/2400 [==============================] - 0s 71us/step - loss: 0.3729 - acc: 0.8608 - val_loss: 0.3763 - val_acc: 0.8483
Epoch 7/30
2400/2400 [==============================] - 0s 77us/step - loss: 0.3521 - acc: 0.8658 - val_loss: 0.3474 - val_acc: 0.8600
Epoch 8

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 1s 382us/step - loss: 0.4375 - acc: 0.8508 - val_loss: 0.4212 - val_acc: 0.8417
Epoch 2/30
2400/2400 [==============================] - 0s 66us/step - loss: 0.4184 - acc: 0.8588 - val_loss: 0.4544 - val_acc: 0.8467
Epoch 3/30
2400/2400 [==============================] - 0s 66us/step - loss: 0.4098 - acc: 0.8583 - val_loss: 0.4133 - val_acc: 0.8417
Epoch 4/30
2400/2400 [==============================] - 0s 65us/step - loss: 0.3914 - acc: 0.8563 - val_loss: 0.4461 - val_acc: 0.8417
Epoch 5/30
2400/2400 [==============================] - 0s 68us/step - loss: 0.4026 - acc: 0.8588 - val_loss: 0.3977 - val_acc: 0.8417
Epoch 6/30
2400/2400 [==============================] - 0s 73us/step - loss: 0.3815 - acc: 0.8588 - val_loss: 0.3844 - val_acc: 0.8467
Epoch 7/30
2400/2400 [==============================] - 0s 68us/step - loss: 0.3815 - acc: 0.8583 - val_loss: 0.3843 - val_acc: 0.8467
Epoch 8

 [0.]]
pod 1st:  0.041666666666666664
Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 1s 426us/step - loss: 0.4528 - acc: 0.8583 - val_loss: 0.4446 - val_acc: 0.8417
Epoch 2/30
2400/2400 [==============================] - 0s 71us/step - loss: 0.4281 - acc: 0.8583 - val_loss: 0.4477 - val_acc: 0.8417
Epoch 3/30
2400/2400 [==============================] - 0s 66us/step - loss: 0.4176 - acc: 0.8583 - val_loss: 0.4408 - val_acc: 0.8417
Epoch 4/30
2400/2400 [==============================] - 0s 66us/step - loss: 0.4170 - acc: 0.8583 - val_loss: 0.4447 - val_acc: 0.8417
Epoch 5/30
2400/2400 [==============================] - 0s 69us/step - loss: 0.4055 - acc: 0.8588 - val_loss: 0.4328 - val_acc: 0.8417
Epoch 6/30
2400/2400 [==============================] - 0s 69us/step - loss: 0.4038 - acc: 0.8588 - val_loss: 0.4326 - val_acc: 0.8417
Epoch 7/30
2400/2400 [==============================] - 0s 66us/step - loss: 0.3975 - acc: 0.8604 - val_

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 1s 429us/step - loss: 0.4625 - acc: 0.8504 - val_loss: 0.4918 - val_acc: 0.8233
Epoch 2/30
2400/2400 [==============================] - 0s 74us/step - loss: 0.4119 - acc: 0.8629 - val_loss: 0.4687 - val_acc: 0.8233
Epoch 3/30
2400/2400 [==============================] - 0s 70us/step - loss: 0.4194 - acc: 0.8629 - val_loss: 0.4707 - val_acc: 0.8233
Epoch 4/30
2400/2400 [==============================] - 0s 70us/step - loss: 0.3968 - acc: 0.8629 - val_loss: 0.4554 - val_acc: 0.8233
Epoch 5/30
2400/2400 [==============================] - 0s 75us/step - loss: 0.3868 - acc: 0.8629 - val_loss: 0.4483 - val_acc: 0.8233
Epoch 6/30
2400/2400 [==============================] - 0s 72us/step - loss: 0.3823 - acc: 0.8629 - val_loss: 0.4299 - val_acc: 0.8233
Epoch 7/30
2400/2400 [==============================] - 0s 73us/step - loss: 0.3788 - acc: 0.8629 - val_loss: 0.4246 - val_acc: 0.8233
Epoch 8

2400/2400 [==============================] - 1s 449us/step - loss: 0.4276 - acc: 0.8629 - val_loss: 0.4786 - val_acc: 0.8233
Epoch 2/30
2400/2400 [==============================] - 0s 76us/step - loss: 0.4162 - acc: 0.8629 - val_loss: 0.4825 - val_acc: 0.8233
Epoch 3/30
2400/2400 [==============================] - 0s 71us/step - loss: 0.4054 - acc: 0.8629 - val_loss: 0.4831 - val_acc: 0.8233
Epoch 4/30
2400/2400 [==============================] - 0s 71us/step - loss: 0.4002 - acc: 0.8629 - val_loss: 0.4619 - val_acc: 0.8233
Epoch 5/30
2400/2400 [==============================] - 0s 75us/step - loss: 0.3931 - acc: 0.8629 - val_loss: 0.4691 - val_acc: 0.8233
Epoch 6/30
2400/2400 [==============================] - 0s 73us/step - loss: 0.3900 - acc: 0.8629 - val_loss: 0.4577 - val_acc: 0.8233
Epoch 7/30
2400/2400 [==============================] - 0s 71us/step - loss: 0.3822 - acc: 0.8629 - val_loss: 0.4730 - val_acc: 0.8233
Epoch 8/30
2400/2400 [==============================] - 0s 74us/s

In [25]:
#RNN
def get_RNN_Predict(X2_train, X2_test, y2_train, y2_test):
    import pandas as pd
    from keras.models import Sequential
    from keras.layers import Dense,Dropout, LSTM, GRU
    from keras.layers import Embedding
    max_features = 10000 # number of words to consider as features
    import numpy as np
    #create model 
    model = Sequential()
    model.add(Embedding(max_features, 32))
    model.add(LSTM(32))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
    history = model.fit(X2_train, y2_train, epochs=5, batch_size=128, validation_split=0.2)
    y2_pred = model.predict(X2_test)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_RNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)
 

Train on 2399 samples, validate on 600 samples
Epoch 1/5
2399/2399 [==============================] - 2s 725us/step - loss: 0.5320 - acc: 0.8579 - val_loss: 0.4307 - val_acc: 0.8417
Epoch 2/5
2399/2399 [==============================] - 0s 83us/step - loss: 0.3988 - acc: 0.8587 - val_loss: 0.4245 - val_acc: 0.8417
Epoch 3/5
2399/2399 [==============================] - 0s 90us/step - loss: 0.3920 - acc: 0.8587 - val_loss: 0.4170 - val_acc: 0.8417
Epoch 4/5
2399/2399 [==============================] - 0s 87us/step - loss: 0.3851 - acc: 0.8587 - val_loss: 0.4134 - val_acc: 0.8417
Epoch 5/5
2399/2399 [==============================] - 0s 86us/step - loss: 0.3805 - acc: 0.8587 - val_loss: 0.4221 - val_acc: 0.8417
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 

Train on 2400 samples, validate on 600 samples
Epoch 1/5
2400/2400 [==============================] - 2s 888us/step - loss: 0.5335 - acc: 0.8250 - val_loss: 0.4324 - val_acc: 0.8417
Epoch 2/5
2400/2400 [==============================] - 0s 84us/step - loss: 0.3996 - acc: 0.8583 - val_loss: 0.4257 - val_acc: 0.8417
Epoch 3/5
2400/2400 [==============================] - 0s 85us/step - loss: 0.3930 - acc: 0.8583 - val_loss: 0.4237 - val_acc: 0.8417
Epoch 4/5
2400/2400 [==============================] - 0s 91us/step - loss: 0.3868 - acc: 0.8583 - val_loss: 0.4175 - val_acc: 0.8417
Epoch 5/5
2400/2400 [==============================] - 0s 105us/step - loss: 0.3801 - acc: 0.8583 - val_loss: 0.4175 - val_acc: 0.8417
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]


Train on 2400 samples, validate on 600 samples
Epoch 1/5
2400/2400 [==============================] - 2s 1ms/step - loss: 0.5471 - acc: 0.8217 - val_loss: 0.4336 - val_acc: 0.8417
Epoch 2/5
2400/2400 [==============================] - 0s 87us/step - loss: 0.4021 - acc: 0.8583 - val_loss: 0.4312 - val_acc: 0.8417
Epoch 3/5
2400/2400 [==============================] - 0s 86us/step - loss: 0.3967 - acc: 0.8583 - val_loss: 0.4258 - val_acc: 0.8417
Epoch 4/5
2400/2400 [==============================] - 0s 86us/step - loss: 0.3950 - acc: 0.8583 - val_loss: 0.4269 - val_acc: 0.8417
Epoch 5/5
2400/2400 [==============================] - 0s 84us/step - loss: 0.3872 - acc: 0.8583 - val_loss: 0.4155 - val_acc: 0.8417
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0

Train on 2400 samples, validate on 600 samples
Epoch 1/5
2400/2400 [==============================] - 3s 1ms/step - loss: 0.5209 - acc: 0.8546 - val_loss: 0.4588 - val_acc: 0.8233
Epoch 2/5
2400/2400 [==============================] - 0s 89us/step - loss: 0.3935 - acc: 0.8629 - val_loss: 0.4636 - val_acc: 0.8233
Epoch 3/5
2400/2400 [==============================] - 0s 90us/step - loss: 0.3872 - acc: 0.8629 - val_loss: 0.4459 - val_acc: 0.8233
Epoch 4/5
2400/2400 [==============================] - 0s 88us/step - loss: 0.3814 - acc: 0.8629 - val_loss: 0.4388 - val_acc: 0.8233
Epoch 5/5
2400/2400 [==============================] - 0s 87us/step - loss: 0.3777 - acc: 0.8629 - val_loss: 0.4467 - val_acc: 0.8233
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0